In [1]:
from bs4 import BeautifulSoup
import requests
import re
from io import BytesIO
from django.core.files.base import File

In [2]:
url = "http://www.annonce-algerie.com/upload/flux/rss_1.xml"

In [3]:
response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
data =response.text 

In [6]:
soup = BeautifulSoup(data,'xml')

In [7]:
items =soup.find_all('item')

In [8]:
links=[]
for item in items:
    links.append(item.find('link').text)

In [9]:
data_ai=[]
for link in links : 
    ai={}
    response = requests.get(link)
    data = BeautifulSoup(response.text,'html.parser')
    tables=data.find_all('table',{'class':'da_rub_cadre'})
    for table in tables :
        table_label=table.find_all('td',{'class':'da_label_field'})
        table_content=table.find_all('td',{'class':'da_field_text'})
        cpt = 0
        if len(table_label)==len(table_content):
            titres = table.find('tr',{'class':'da_entete'})
            if titres!=None:
                titre =titres.text  
                titre_table =titre.split(' ')
                co = 1 
                titre=''
                while len(titre_table) >co :
                    titre = titre+' '+titre_table[co]
                    co=co+1
                ai['titre']=titre
                while cpt < len(table_label) :
                    if table_label:
                        if table_label[cpt].text.strip()=='Catégorie' :
                            label=table_label[cpt].text
                            content=table_content[cpt].text
                            contents=content.split('>')
                            if len(contents)>2:
                                content = contents[1].strip()
                            ai[label]= content
                        elif table_label[cpt].text.strip()=='Localisation' :
                            label=table_label[cpt].text
                            content=table_content[cpt].text
                            contents=content.split('>')
                            if len(contents)>2:
                                content = contents[len(contents)-2].strip()+" "+contents[len(contents)-1].strip()
                            ai[label]=content
                        elif table_label[cpt].text.strip()=='Modifiée le':
                                    ai['date_Publication']=table_content[cpt].text.replace("/", "-")[::-1]+'T00:00'
                        elif table_label[cpt].text.strip()=='Prix'or table_label[cpt].text.strip()=='Surface': 
                            label=table_label[cpt].text.strip()
                            content= re.findall(r'\d+',table_content[cpt].text.strip())[0]
                            ai[label]=content
                        else    :
                            label=table_label[cpt].text.strip()
                            content=table_content[cpt].text.strip()
                            ai[label]=content
                    cpt=cpt+1
        imgs=table.find_all('img',{'class':'PhotoMin1'})
        images=[]
        if imgs !=None :
            for img in imgs:
                image_url= "http://www.annonce-algerie.com/"+img['src']
                response = requests.get(image_url)
                file = BytesIO(response.content)
                file_ai=File(file)
                images.append(file_ai)
            if len(images)!= 0 : 
                ai['uploaded_images']=images    
    data_tel = data.find('li',{'class':'cellphone'})
    tel = data_tel.text.split(':')[1].strip()
    ai['information_tel']=tel
    data_ai.append(ai)       

In [10]:
rel_data_ai = []
for data in data_ai:
    if len(data) > 1:
        if not ('uploaded_images' in data):
            data['uploaded_images'] = []
        data['user'] = 1
        rel_data_ai.append(data)

In [11]:
print(rel_data_ai)

[{'titre': ' Photos', 'Catégorie': 'Terrain', 'Localisation': 'Djelfa Ain Oussera', 'Adresse': 'a.oussera', 'Surface': '500', 'Prix': '850', 'Texte': 'terrain a vendre de 50 h ( livret fonçier+ forage+ l electricite a cote) +\r110h ( livret fonçier equiper de tout ) pour plus d informations apllez moi et merci \r0781645418/0775168519/0668620144', 'Insérée le': '01/02/2023', 'date_Publication': '3202-20-10T00:00', 'uploaded_images': [<File: None>, <File: None>, <File: None>], 'information_tel': '0668620144', 'user': 1}, {'titre': ' 700m commercial café chergui vert alger', 'Catégorie': 'Vente', 'Localisation': 'Alger Bordj El Kiffan', 'Adresse': 'café chergui vert alger', 'Surface': '700', 'Prix': '20', 'Texte': 'agence immobilière ilya vd commercial 100% terrain deux 02façade 700m² acte et livret de foncier le prix : 14 milliards  tél ou wathssap 213 770 407 047', 'Insérée le': '01/02/2023', 'date_Publication': '3202-20-10T00:00', 'information_tel': '+213 77 04 07 047', 'uploaded_image

In [17]:
import requests
import magic
from io import BytesIO
response = requests.get('https://picsum.photos/id/237/200/300')

file_stream = BytesIO(response.content)
magic_obj = magic.Magic(mime=True)

# Get the MIME type of the file
file_type = magic_obj.from_buffer(file_stream.getvalue())
name_image = "scrapping."+file_type.split('/')[1]
print(name_image)
print(file_type)

scrapping.jpeg
image/jpeg


In [14]:
!pip install python-magic

Defaulting to user installation because normal site-packages is not writeable
